In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

## Aprendizado não supervisionado

In [ ]:
df = pd.read_csv('../input/spotifyclassification/data.csv', index_col=0)

### O foco da seção de aprendizado não supervisionado é agregar músicas em grupos que representariam seus gêneros baseadas em diferentes características. O objetivo final é encontrar a medida ótima para o número de clusters utilizando o coeficiente de silhueta.
### Essa métrica permite avaliar o quão similar é um sample ao seu próprio cluster comparado com o restante dos clusters. Em suma, o coefiente indica o quão próximo os pontos de um cluster se encontram de pontos em um cluster vizinho. Sua fórmula é simples. Calcula-se a média da distância intra-cluster para cada agrupamento (mi) e a média das distâncias de cada sample ao cluster mais próximo (mn). O resultado final é: (mn - mi)/ max(mi, mn).
### Um coeficiente próximo de 1 indica que o sample está alonge de clusters vizinhos, ao passo que um valor de 0 mostra que o sample está na fronteira de um cluster e outros. Um valor de -1 resulta de samples atribuídos ao cluster errado. Por meio desses valores, pode se selecionar o melhor número de clusters possíveis assim como avaliar se os dados realmente têm características que permite o agrupamento em diversos clusters.


In [ ]:
df


In [ ]:
# Remover colunas com o nome da música e do artista, assim como a label de cada música
df.drop(columns=["target", "song_title", "artist"],  inplace=True)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples

 

n_clusters_range = range(2, 15)
silhouetes_scrs = []
silhouetes_scrs_samples = []
for n_clusters in n_clusters_range:
    kmeans = KMeans(n_clusters=n_clusters).fit(df)
    silhouete_scr = silhouette_score(df, kmeans.labels_)
    silhouetes_scrs.append(silhouete_scr)
    silhouetes_scrs_samples.append(silhouette_samples(df, kmeans.labels_))
 
    

In [ ]:
silhouetes_scrs

In [ ]:
import matplotlib.pyplot as plt

plt.plot(n_clusters_range, silhouetes_scrs)
plt.xlabel("Número de clusters")
plt.ylabel("Coeficiente de silhueta")
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=2).fit(df)

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2,  n_iter=1000, init='pca').fit_transform(df)


In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(tsne[:,0], tsne[:,1], c=kmeans.labels_);

## Aprendizado supervisionado

In [ ]:
pitchfork_df= pd.read_csv('../input/pitchfork-reviews-through-12617/p4kreviews.csv',encoding='latin-1', index_col=0 )

In [ ]:
pitchfork_df

### O foco da seção de aprendizado supervisionado é criar um regressor que baseado textos de críticos que avaliam álbuns na plataforma de música [Pitchfork](https://pitchfork.com/) retorna um valor de 0 a 10 representando a nota dada ao disco.

### Para avaliar a performance do modelo, o r2 score será utilizado. Essa métrica, que normalmente varia de 0 a 1, permite verificar o quanto da variância da variável dependente (o y, nesse caso os scores dados pelos críticos) que pode ser explicado pelas váriáveis dependentes, nesse caso a matrix com a vetorização dos textos.

### Seu cálculo é simples: é dado por 1 menos a soma do quadrado dos resíduos, ou seja, a soma das diferenças dos y's originais com os y's preditos ao quadrado, divido pela soma total dos quadrados, que é a soma dos quadrados das diferença de cada y original com a média dos y's.

### Pré-processamento + split

In [ ]:
pitchfork_df = pitchfork_df[["review", "score"]]

In [ ]:
pitchfork_df.info()

In [ ]:
pitchfork_df.dropna(inplace=True)

In [ ]:
pitchfork_df.info()

In [ ]:
import nltk
import re
stopwords = list(nltk.corpus.stopwords.words('english'))

def preprocessing(text):
    regex = re.compile('[^a-zA-Z\s]+')
    text = text.lower()
    text = regex.sub('', text)
    
    words = text.split()
    words = [word for word in words if word not in stopwords]
    
    return ' '.join(words)
    

In [ ]:
pitchfork_df["review"] = pitchfork_df.review.apply(preprocessing)

In [ ]:
pitchfork_df

In [ ]:
import matplotlib.pyplot as plt

plt.hist(pitchfork_df.score)
plt.show()

In [ ]:
# Criar 10 bins para estratificar coluna dos scores (ref: https://michaeljsanders.com/2017/03/24/stratify-continuous-variable.html)
bins = np.linspace(0, len(pitchfork_df), 10)
y_binned = np.digitize(pitchfork_df.score, bins)

In [ ]:
from sklearn.model_selection import train_test_split

X, y = pitchfork_df.review, pitchfork_df.score

X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y_binned, random_state=42)

### Treinamento + Predição

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer
from xgboost import XGBRegressor

cls = XGBRegressor().fit(HashingVectorizer(n_features=2**8).fit_transform(X_train), y_train)


In [ ]:
y_pred = cls.predict(HashingVectorizer(n_features=2**8).fit_transform(X_test))

In [ ]:
from sklearn.metrics import r2_score

r2_score(y_test, y_pred)